## Export All Data to Single CSV
This notebook will walk you through the process of exporting indicator data as a single CSV file

In [ ]:
import os
import pyodbc
import csv
from IPython.display import display
from ipywidgets import widgets, Layout, Label, VBox

# [x for x in pyodbc.drivers() if x.startswith('Microsoft Access Driver')]
# if you see [], you may need to install the 64-bit ACE drivers
# https://github.com/mkleehammer/pyodbc/wiki/Connecting-to-Microsoft-Access
# 64bit drivers : https://www.microsoft.com/en-us/download/confirmation.aspx?id=13255

## Set defaults

### Where is your working folder? usually just the name of the country you are working on

In [ ]:
output_base = None

lbl_out = Label('Where is your working folder? This is usually just the name of your country.')
output_base_txt = widgets.Text(placeholder='my_country_name', 
                  layout=Layout(width='50%'))

out = widgets.Output()
def on_value_change(change):
    with out:
        global output_base
        output_base = change['new']

output_base_txt.observe(on_value_change, names='value')

display(VBox([lbl_out, output_base_txt, out]))

### Enter the path to your DevInfo Access Database

In [ ]:
access_database = None

lbl_adb = Label('Enter the Path to your DevInfo Access Database')
access_database_txt = widgets.Text(placeholder='ex: C:\\users\\me\\devinfo\\devinfo.accdb (.mdb)', 
                  layout=Layout(width='80%'))

out = widgets.Output()
def on_value_change(change):
    with out:
        global access_database
        access_database = change['new']

access_database_txt.observe(on_value_change, names='value')

display(VBox([lbl_adb, access_database_txt, out]))

### Enter the name of your output CSV file

In [ ]:
output_csv = None

lbl_csv = Label('Name your output CSV file')
output_csv_txt = widgets.Text(placeholder='ex: devinfo_output.csv', 
                  layout=Layout(width='80%'))

out = widgets.Output()
def on_value_change(change):
    with out:
        global output_csv
        output_csv = change['new']

output_csv_txt.observe(on_value_change, names='value')

display(VBox([lbl_csv, output_csv_txt, out]))

In [ ]:
access_database = r'C:\Users\adam\devinfo\tanzania\TSED_20180423.mdb'
output_csv = 'devinfo_output_tz.csv'

## Execute the query against the DevInfo Access tables

In [ ]:
connStr = (
    r"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};"
    r"DBQ={};".format(access_database)
    )

cnxn = pyodbc.connect(connStr)

sql = """\
SELECT
UT_Data.Indicator_NId AS INDICATOR_ID, 
UT_Indicator_en.Indicator_Name AS INDICATOR, 
UT_Data.Data_Value AS OBS_VALUE, 
UT_Unit_en.Unit_NId AS UNIT_ID, 
UT_Unit_en.Unit_Name AS UNIT, 
UT_Area_en.Area_ID AS REF_AREA_ID, 
UT_Area_en.Area_Name AS REF_AREA, 
UT_TimePeriod.TimePeriod AS TIME_PERIOD,

UT_Indicator_Classifications_en.Publisher AS PUBLISHER,
UT_Area_Map_Layer.Layer_NId AS LAYER_ID,

UT_Area_Level_en.Area_Level_Name AS AREA_LEVEL_NAME, 
UT_Area_en.Area_Level AS AREA_LEVEL,
UT_Indicator_Classifications_en.IC_Name AS IC_NAME, 
UT_Subgroup_Vals_en.Subgroup_Val AS SUBGROUP_VAL, 
UT_Subgroup_Type_en.Subgroup_Type_Name AS SUBGROUP_TYPE_NAME

FROM 
((((UT_Area_Map_Layer INNER JOIN ((UT_Area_Level_en INNER JOIN (UT_Subgroup_Vals_en INNER JOIN (UT_Unit_en INNER JOIN (UT_Indicator_en INNER JOIN (UT_Indicator_Unit_Subgroup INNER JOIN (UT_TimePeriod INNER JOIN (UT_Indicator_Classifications_en INNER JOIN (UT_Area_en INNER JOIN UT_Data ON UT_Area_en.[Area_NId] = UT_Data.[Area_NId]) ON UT_Indicator_Classifications_en.IC_NId = UT_Data.Source_NId) ON UT_TimePeriod.TimePeriod_NId = UT_Data.TimePeriod_NId) ON UT_Indicator_Unit_Subgroup.IUSNId = UT_Data.IUSNId) ON UT_Indicator_en.Indicator_NId = UT_Indicator_Unit_Subgroup.Indicator_NId) ON UT_Unit_en.Unit_NId = UT_Indicator_Unit_Subgroup.Unit_NId) ON UT_Subgroup_Vals_en.Subgroup_Val_NId = UT_Indicator_Unit_Subgroup.Subgroup_Val_NId) ON UT_Area_Level_en.Area_Level = UT_Area_en.Area_Level) INNER JOIN UT_Area_Map ON UT_Area_en.Area_NId = UT_Area_Map.Area_NId) ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map.Layer_NId) INNER JOIN UT_Area_Map_Metadata_en ON UT_Area_Map_Layer.Layer_NId = UT_Area_Map_Metadata_en.Layer_NId) INNER JOIN UT_Subgroup_Vals_Subgroup ON UT_Subgroup_Vals_en.Subgroup_Val_NId = UT_Subgroup_Vals_Subgroup.Subgroup_Val_NId) INNER JOIN UT_Subgroup_en ON UT_Subgroup_Vals_Subgroup.Subgroup_NId = UT_Subgroup_en.Subgroup_NId) INNER JOIN UT_Subgroup_Type_en ON UT_Subgroup_en.Subgroup_Type = UT_Subgroup_Type_en.Subgroup_Type_NId

ORDER BY 
UT_Data.Indicator_NId
"""

crsr = cnxn.execute(sql)

rows = crsr.fetchall()

print ('sucessfully executed data query :: {} rows returned'.format(len(rows)))

## Prepare to query the DevInfo Access Database
Here we will map our DevInfo fields to the DSD as defined here. **TODO :: add link(s) to reference data schema**

This is our ouptut data schema that will be in the CSV file.

In [ ]:
# 'DSD_FIELD' : 'DevInfoField'
field_mappings = [
    'INDICATOR_ID',
    'INDICATOR',
    'REF_AREA',
    'REF_AREA_ID',
    'OBS_VALUE',
    'UNIT_ID',
    'UNIT',
    'TIME_PERIOD'
]

# add in any additional fields you want in the output
add_fields = [
    'PUBLISHER',
    'LAYER_ID'
]

# combine the two dictionaries
field_mappings = field_mappings + add_fields

## Write the result to a CSV file

In [ ]:
output_path = os.path.join(output_base, output_csv)

# set encoding 
# when reading some CSV files, there may be encoding issues that result in unicode characters appearing in the field names
# if you have any unexpected behavior that is related to this, either re-save your file with "utf-8" encoding
# or try using 'utf-8-sig' as your encoding value below
# Stack Overflow reference: https://stackoverflow.com/questions/17912307/u-ufeff-in-python-string/17912811#17912811
encoding = 'utf-8'
with open(output_path, 'w', encoding=encoding, newline='') as csvfile:
    writer = csv.writer(csvfile)
    
    header_row = []
    for field in field_mappings:
        header_row.append(field)
        
    writer.writerow(header_row)
    
    for row in rows:
        new_row = []
        for field in field_mappings:
            new_row.append(getattr(row, field))
        
        writer.writerow(new_row)

print ('csv successfully created at {}'.format(os.path.abspath(output_path)))